In [1]:
import os
import requests as req
import seaborn as sns
import hypertools as hyp
import pandas as pd
import numpy as np
from helpers import *
import html2text
import markdown
from bs4 import BeautifulSoup
from scipy.spatial.distance import cdist, pdist, squareform
import nltk
nltk.download('stopwords')
import matplotlib.pyplot as plt

from nltk.corpus import stopwords as sw

%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
fileid = '1hCCn31z4HM4IzQi59DP-vvUpYKhlvo2S'
datadir = '../data'
fname = os.path.join(datadir, 'data.csv')

if not os.path.exists(datadir):
    os.makedirs(datadir)
    
if not os.path.exists(fname):
    print('Fetching file from the interwebz!')
    dl(fileid, fname)
else:
    print('Data found!')

Data found!


In [3]:
data = pd.read_csv(fname)

In [4]:
data.head()

,id,title,url,writers,genres,script
0,1,10 Things I Hate About You,http://www.imsdb.com/scripts/10-Things-I-Hate-...,Karen McCullah Lutz;Kirsten Smith;William Shak...,Comedy;Romance,\n\n\n<b><!--\n</b>if (window!= top)\ntop.loca...
1,2,12,http://www.imsdb.com/scripts/12.html,Lawrence Bridges,Comedy,\n\n \n 12 - Script\n
2,3,12 and Holding,http://www.imsdb.com/scripts/12-and-Holding.html,Anthony Cipriano,Drama,\n \n \n ...
3,4,12 Monkeys,http://www.imsdb.com/scripts/12-Monkeys.html,David Peoples;Janet Peoples,Drama;Sci-Fi;Thriller,\n\n\n<b><!--\n</b>if (window!= top)\ntop.loca...
4,5,12 Years a Slave,http://www.imsdb.com/scripts/12-Years-a-Slave....,John Ridley,Drama,\r\n\r\n\r\n 12 YE...


## Add review filenames

In [29]:
filenames = ['../../aclImdb/test/urls_neg.txt',
            '../../aclImdb/test/urls_pos.txt',
            '../../aclImdb/train/urls_neg.txt',
            '../../aclImdb/train/urls_pos.txt']
sets = ['test','test','train','train']
sentiments = ['neg','pos','neg','pos']
review_data = pd.DataFrame()
for fi in [0,1,2,3]:
    set_cur = sets[fi]
    sentiment_cur = sentiments[fi]
    print((set_cur,sentiment_cur))
    filename = '../../aclImdb/%s/urls_%s.txt'%(set_cur,sentiment_cur)
    tmp = pd.read_csv(filename,header=None)
    tmp.columns = ['url']
    for i in tmp.index:
        tmp.loc[i,'imdb_id'] = tmp.loc[i,'url'][28:35]
    tmp['set'] = set_cur
    tmp['sentiment'] = sentiment_cur
    tmp['review_id'] = tmp.index
    review_data = review_data.append(tmp)
review_data = review_data.reset_index(drop=True)
review_data.head()

('test', 'neg')
('test', 'pos')
('train', 'neg')
('train', 'pos')


,url,imdb_id,set,sentiment,review_id
0,http://www.imdb.com/title/tt0406816/usercomments,0406816,test,neg,0
1,http://www.imdb.com/title/tt0406816/usercomments,0406816,test,neg,1
2,http://www.imdb.com/title/tt0406816/usercomments,0406816,test,neg,2
3,http://www.imdb.com/title/tt0406816/usercomments,0406816,test,neg,3
4,http://www.imdb.com/title/tt0406816/usercomments,0406816,test,neg,4


In [33]:
review_data.to_csv('../data/review_data.csv')

In [7]:
data['imdb_id'] = np.nan
to_do = []
for i in np.arange(0,len(data)):
    if not type(data.loc[i,'imdb_id']) == np.str:
        if np.isnan(data.loc[i,'imdb_id']):
            to_do.append(i)

In [8]:
print(np.array(to_do))

[   0    1    2 ... 1115 1116 1117]


In [10]:
from imdb import IMDb

In [11]:
imdb_instance = IMDb()
for mi,movie in data.iloc[to_do,:].iterrows():
    title = movie['title']
    imdb_searchresult = imdb_instance.search_movie(title,1)
    if len(imdb_searchresult) > 0:
        imdb_info = imdb_searchresult[0]
        imdb_id = imdb_info.getID()
        data.loc[mi,'imdb_id'] = imdb_id
data.head()

,id,title,url,writers,genres,script,imdb_id
0,1,10 Things I Hate About You,http://www.imsdb.com/scripts/10-Things-I-Hate-...,Karen McCullah Lutz;Kirsten Smith;William Shak...,Comedy;Romance,\n\n\n<b><!--\n</b>if (window!= top)\ntop.loca...,0147800
1,2,12,http://www.imsdb.com/scripts/12.html,Lawrence Bridges,Comedy,\n\n \n 12 - Script\n,1407084
2,3,12 and Holding,http://www.imsdb.com/scripts/12-and-Holding.html,Anthony Cipriano,Drama,\n \n \n ...,0417385
3,4,12 Monkeys,http://www.imsdb.com/scripts/12-Monkeys.html,David Peoples;Janet Peoples,Drama;Sci-Fi;Thriller,\n\n\n<b><!--\n</b>if (window!= top)\ntop.loca...,0114746
4,5,12 Years a Slave,http://www.imsdb.com/scripts/12-Years-a-Slave....,John Ridley,Drama,\r\n\r\n\r\n 12 YE...,2024544


In [12]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [22]:
has_reviews = intersection(review_data['imdb_id'].unique(),data['imdb_id'].unique())

In [23]:
np.array(has_reviews)

array(['0110604', '0349903', '0808151', '0120907', '0062622', '0171580',
       '0209958', '0070337', '0317740', '0116209', '0159365', '0151738',
       '0361748', '0166924', '0113253', '0100260', '0118749', '0455857',
       '0238546', '0119925', '0375679', '0199725', '0077405', '0090329',
       '0144814', '0120591', '0963794', '0159382', '0162661', '0110265',
       '0317640', '0074812', '0913425', '0069945', '0134273', '0159206',
       '0333766', '0119217', '0120737', '0318374', '0101393', '0387564',
       '0104691', '0134119', '0417217', '0118158', '0379725', '0113627',
       '0163025', '0120812', '0088323', '0165710', '0100405', '0109450',
       '0460627', '0116367', '0271367', '0119229', '0078788', '0044079',
       '0116908', '0462200', '0103776', '0049470', '0092699', '0086190',
       '0164052', '0093565', '0082031', '0926129', '0081505', '0440963',
       '0887971', '0187078', '0132347', '0259711', '0106834', '0328107',
       '0107302', '0087957', '0455538', '0091969', 

In [24]:
data['has_review'] = False
data.loc[data['imdb_id'].isin(has_reviews),'has_review'] = True
data.head()

,id,title,url,writers,genres,script,imdb_id,has_review
0,1,10 Things I Hate About You,http://www.imsdb.com/scripts/10-Things-I-Hate-...,Karen McCullah Lutz;Kirsten Smith;William Shak...,Comedy;Romance,\n\n\n<b><!--\n</b>if (window!= top)\ntop.loca...,0147800,False
1,2,12,http://www.imsdb.com/scripts/12.html,Lawrence Bridges,Comedy,\n\n \n 12 - Script\n,1407084,False
2,3,12 and Holding,http://www.imsdb.com/scripts/12-and-Holding.html,Anthony Cipriano,Drama,\n \n \n ...,0417385,False
3,4,12 Monkeys,http://www.imsdb.com/scripts/12-Monkeys.html,David Peoples;Janet Peoples,Drama;Sci-Fi;Thriller,\n\n\n<b><!--\n</b>if (window!= top)\ntop.loca...,0114746,True
4,5,12 Years a Slave,http://www.imsdb.com/scripts/12-Years-a-Slave....,John Ridley,Drama,\r\n\r\n\r\n 12 YE...,2024544,False


In [25]:
data.to_csv('../data/data_with_imdb_info.csv')